In [1]:
import sqlite3
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

In [2]:
def parse_score(score_as_string):
        if score_as_string is None: 
            return None
        home_goals, away_goals = score_as_string.split(":")
        home_goals = int(home_goals)
        away_goals = int(away_goals)
        if home_goals > away_goals: 
            return "1"
        if away_goals > home_goals: 
            return "2" 
        else: 
            return "X"
        
def goals_dif_home(score_as_string):
        home_goals, away_goals = score_as_string.split(":")
        home_goals = int(home_goals)
        away_goals = int(away_goals)
        return (home_goals - away_goals)
    
def goals_dif_away(score_as_string):
        away_goals, home_goals = score_as_string.split(":")
        home_goals = int(home_goals)
        away_goals = int(away_goals)
        return (home_goals - away_goals)
    


In [3]:
with sqlite3.connect("../laliga.sqlite") as conn:
    df = pd.read_sql("SELECT * FROM Matches", con = conn)

dataset = df.loc[-df.score.isnull()]

dataset['home_GD_lm'] = dataset['score'].apply(goals_dif_home)
dataset['away_GD_lm'] = dataset['score'].apply(goals_dif_away)

dataset['result'] = dataset['score'].apply(parse_score)

# convert strings in numbers 
# ord_enc = OrdinalEncoder()
# dataset["home_team"] = ord_enc.fit_transform(dataset[["home_team"]])
# dataset["away_team"] = ord_enc.fit_transform(dataset[["away_team"]])
# dataset['result'] = ord_enc.fit_transform(dataset[['result']])

dataset['season2'] = dataset['season']
dataset = dataset.astype({"season": str})
dataset[['season','season2']] = dataset['season'].str.split("-",expand=True)
dataset = dataset.astype({"season": int})
dataset = dataset.drop('season2', axis=1)
dataset = dataset.drop(['date','time','score'], axis=1)

import sys
import warnings
warnings.filterwarnings("ignore")

dataset.head(20)

C:\Users\jorge\AppData\Local\Temp/ipykernel_5148/2659846943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['home_GD_lm'] = dataset['score'].apply(goals_dif_home)
C:\Users\jorge\AppData\Local\Temp/ipykernel_5148/2659846943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['away_GD_lm'] = dataset['score'].apply(goals_dif_away)
C:\Users\jorge\AppData\Local\Temp/ipykernel_5148/2659846943.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,season,division,matchday,home_team,away_team,home_GD_lm,away_GD_lm,result
0,1928,1,1,Arenas Club,Athletic Madrid,-1,1,2
1,1928,1,1,Espanyol,Real Unión,1,-1,1
2,1928,1,1,Real Madrid,Catalunya,5,-5,1
3,1928,1,1,Donostia,Athletic,0,0,X
4,1928,1,1,Racing,Barcelona,-2,2,2
5,1928,1,2,Barcelona,Real Madrid,-1,1,2
6,1928,1,2,Athletic,Espanyol,9,-9,1
7,1928,1,2,Athletic Madrid,Donostia,-3,3,2
8,1928,1,2,Real Unión,Racing,2,-2,1
9,1928,1,2,Catalunya,Arenas Club,3,-3,1


In [5]:
import openpyxl

# Save as an excel file
dataset.to_csv(r'..\reports\Prueba.csv')